# Entrega 1 Optimización

### Grupo 4

Integrantes:
* Carlos Arevalo
* Nangel Coello
* Daniel Maturana
* Benjamin Pavez

## Problema de Asignación

### Enunciado
En una empresa se busca asignar de forma óptima a sus trabajadores en las diversas tareas que deben llevarse a cabo. Se sabe que los trabajadores están clasificados según su nivel de especialización. Además, las tareas están clasificadas según los mismos niveles de especialización de tal manera que una tarea de cierto nivel de especialización puede ser ejecutada solo por trabajadores de tal nivel, o bien, uno superior, pero no inferior.

Cada tarea requiere de cierta cantidad de tiempo para ser completada y cada trabajador puede ser asignado por una cantidad determinada de tiempo. Todas las tareas deben ser completadas en su totalidad.

Todos los trabajadores de cierto nivel de especialización suponen un costo por unidad de tiempo y un costo fijo asociado a asignar a cierto trabajador a cierta tarea en específico. A estos costos los llamaremos $costos$ $1$. Además, existe un costo por sobrecalificación asociada a cada tarea.

Cada trabajador puede ser asignado a máximo \$N\$ tareas, pero para completar una tarea puede requerirse más de un trabajador. No obstante, no se pueden asignar más de dos tercios del total de los trabajadores a solo una tarea.

La empresa cuenta con un presupuesto \$P\$ para la asignación de tareas que no puede ser superado, y busca:
1. Minimizar $costos$ $1$.
2. Minimizar costos de sobrecalificación.

### Formulación del Modelo Matemático

#### Conjuntos:
- *I* : Conjunto de trabajadores, i $\in$ *I*.
- *J* : Conjunto de tareas, j $\in$ *J*.
- *L* : Conjunto de niveles de especialización,  l $\in$ \{1, 2, 3, $\ldots$, 8\}.

#### Parámetros:
- $ E_i $: Nivel de especialización del trabajador $ i $.
- $ S_j $: Nivel de especialización requerido para la tarea $ j $.
- $ T_j $: Tiempo requerido para completar la tarea $ j $.
- $ H_i $: Tiempo disponible del trabajador $ i $.
- $ C_{ij} $: Costo fijo asociado a asignar al trabajador $ i $ a la tarea $ j $.
- $ U_i $: Costo por unidad de tiempo del trabajador $ i $.
- $ O_{ij} $: Costo por sobrecalificación de asignar al trabajador $ i $ a la tarea $ j $ (si $ E_i > S_j $).
- $ P $: Presupuesto disponible.
- $ N $: Máximo número de tareas a las que puede ser asignado un trabajador.

#### Variables:
- $x_{ij}$ : Variable binaria que indica si el trabajador $i$ es asignado a la tarea $j$.
- $y_{ij}$ : Tiempo que el trabajador $i$ dedica a la tarea $j$.

#### Función objetivo:
Minimizar los costos 1:

$
\text{Minimizar} \quad Z = \sum_{i \in I} \sum_{j \in J} (C_{ij} \cdot x_{ij} + U_i \cdot y_{ij})
$

#### Restricciones:
1. **Asignación completa de tareas**: Cada tarea debe ser completada en su totalidad.
   $
   \sum_{i \in I} y_{ij} = T_j \quad \forall j \in J
   $  

   
2. **Tiempo disponible por trabajador**: Un trabajador no puede ser asignado más allá de su tiempo disponible.
   $
   \sum_{j \in J} y_{ij} \leq H_i \quad \forall i \in I
   $  

  
3. **Nivel de especialización**: Un trabajador puede ser asignado a una tarea solo si su nivel de especialización es igual o superior al requerido.
   $
   x_{ij}( E_i - S_j) \le 0
   $

  
4. **Relación entre $x_{ij}$ y $y_{ij}$** : Si un trabajador $i$ no está asignado a una tarea $j$, el tiempo dedicado es cero.
   $
   y_{ij} \leq T_j \cdot x_{ij} \quad \forall i \in I, \forall j \in J
   $


5. **Límite de tareas por trabajador**: Un trabajador puede ser asignado a un máximo de $N$ tareas.
   $
   \sum_{j \in J} x_{ij} \leq N \quad \forall i \in I
   $


6. **Presupuesto**: El costo total no debe superar el presupuesto disponible.
   $
   \sum_{i \in I} \sum_{j \in J} (C_{ij} \cdot x_{ij} + U_i \cdot y_{ij} + O_{ij} \cdot x_{ij}) \leq P
   $


7. **Límite de asignación por tarea**: No se pueden asignar más de dos tercios de los trabajadores a una sola tarea.
   $
   \sum_{i \in I} x_{ij} \leq \frac{2}{3}|I| \quad \forall j \in J
   $


8. **Naturaleza variables**:
   
   $x_{ij} \in \{0, 1\} \quad \forall i \in I, \forall j \in J$
   
   $y_{ij} \ge 0 \quad \forall i \in I, \forall j \in J$

### Posibles Casos de Infactibilidad

- **Insuficiente tiempo disponible**: Si el tiempo total requerido por todas las tareas es mayor que la suma del tiempo disponible de todos los trabajadores.
- **Presupuesto insuficiente**: Si el presupuesto disponible es menor que el costo mínimo necesario para asignar todas las tareas.
- **Desajuste en niveles de especialización**: Si hay tareas para las cuales no hay trabajadores con un nivel de especialización adecuado.
- **Exceso de tareas por trabajador**: Si las restricciones de un máximo de $N$ tareas por trabajador no pueden cumplirse debido a la cantidad de tareas y trabajadores.
- **Límite de asignación por tarea**: Si algunas tareas requieren más trabajadores de los que pueden ser asignados sin exceder el límite de dos tercios.


### Generador de Instancias en Python

In [16]:
import numpy as np
import random

def generar_instancia(I, J, N):
    # Niveles de especialización (distribución triangular)
    E = np.random.triangular(1, 8, 8, I).astype(int)
    S = np.random.triangular(1, 1, 8, J).astype(int)

    # Tiempo disponible por trabajador y tiempo requerido por tarea
    inf_limit = 20 + 10*N
    sup_limit = inf_limit + 15
    H = np.random.randint(inf_limit, sup_limit, I)*10
    T = np.random.randint(inf_limit, sup_limit, J)*10

    # Costos por unidad de tiempo por trabajador
    costo_tiempo = lambda n_e: np.random.uniform(5*n_e, 5*n_e+5)
    U = costo_tiempo(E)

    # Costos fijos y costos de sobrecalificación
    C = np.random.randint(4000, 8001, (I, J))
    O = np.random.randint(10000, 30001, (I, J))

    # Presupuesto
    P = np.random.randint(2 * (I // 3) * 10000, 4 * (I // 5) * 10000)

    return E, S, H, T, U, C, O, P

# Ejemplo de generación de instancia
I, J, N = 10, 5, 4
E, S, H, T, U, C, O, P = generar_instancia(I, J, N)
print("Niveles de especialización de trabajadores:", E)
print("Niveles de especialización requeridos por tareas:", S)
print("Tiempos disponibles de trabajadores:", H)
print("Tiempos requeridos por tareas:", T)
print("Costos por unidad de tiempo de trabajadores:", U)
print("Costos fijos de asignación:", C)
print("Costos de sobrecalificación:", O)
print("Presupuesto:", P)




Niveles de especialización de trabajadores: [7 3 7 4 6 6 2 7 3 7]
Niveles de especialización requeridos por tareas: [3 7 1 2 4]
Tiempos disponibles de trabajadores: [620 680 740 660 660 630 620 620 670 630]
Tiempos requeridos por tareas: [710 730 680 660 730]
Costos por unidad de tiempo de trabajadores: [36.54763351 17.71099203 37.952245   21.36988357 33.66856743 32.01272211
 12.77080243 39.5430861  16.56543951 37.50085118]
Costos fijos de asignación: [[6087 4395 5666 5523 7164]
 [6648 4100 4851 5216 7957]
 [4347 5748 7493 7510 7916]
 [4222 7680 5564 5501 7179]
 [6209 5980 5510 6895 4978]
 [7079 4109 6631 4647 5523]
 [4144 6746 4683 6931 7269]
 [7616 5199 4619 7559 6304]
 [4575 4809 5921 5569 7268]
 [5472 5577 6628 7258 6674]]
Costos de sobrecalificación: [[24950 16247 19967 12806 21847]
 [11184 16159 27234 16838 18427]
 [22231 26214 26426 23163 17726]
 [25141 15433 21073 28883 26407]
 [22894 27678 29020 18164 22525]
 [14055 23905 18380 15818 19875]
 [18061 26189 22872 15374 17364]
 [2